### Import Libraries

In [ ]:
pip install yfinance

In [ ]:
import os
import dotenv
import yfinance as yf
from pyspark.sql import SparkSession
import pandas as pd

#### Extraction Layer

In [ ]:

# Set Java Environment
os.environ['JAVA_HOME'] = "C:/java8"

# Initialize Spark session
spark = SparkSession.builder.appName("StockData").getOrCreate()

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [ ]:
# dotenv.load_dotenv()

# API_KEY = os.getenv('API_KEY')

# print(API_KEY)

In [ ]:
# Data Ingestion for AlphaVanage
# symbols = ["AAPL, "GOOGL", "MSFT", "AMZN", "TSLA", "IBM"]

# alphavantage_data = {}
# for symbol in symbols:
#     BASE_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={API_KEY}'
#     response = requests.get(BASE_URL)
#     if response.status_code == 200:
#         data = response.json()
#         if "Time Series (Daily)" in data:
#             alphavantage_data[symbol] = data["Time Series (Daily)"]
#             print(f"Fetched data for {symbol}")
#         else:
#             print(f"No data found for {symbol} or API limit reached")
#     else:
#         print(f"Unable to fetch data for {symbol}")

In [ ]:
#Data Ingestion for YahooFinance
tickers = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA", "IBM"]

stock_data = yf.download(tickers, period="1mo", interval="1d", group_by='ticker', threads=True)

# Empty Spark DataFrame to combine all
stock_df = None

# Prepare a list to store dataframes
dataframes = []

# Iterate through the tickers and process the data
for ticker in tickers:
    # Get the data for each ticker
    ticker_data = stock_data[ticker].reset_index()  # Reset index for Date as a column
    ticker_data['Ticker'] = ticker  # Add a 'Ticker' column
    dataframes.append(ticker_data)

# Merge all the dataframes into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Convert the merged pandas dataframe to a PySpark dataframe
spark_df = spark.createDataFrame(merged_df)

# Show the schema and first few rows
spark_df.printSchema()
spark_df.show(5)


In [ ]:
print(data.columns)

#### Transformation Layer